# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [4]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
#!pip install fetch-my-weather
#!pip install hands-on-ai
#!pip install pyinputplus
#!pip install pandas

!pip install pyinputplus requests matplotlib pandas
!pip install fetch-my-weather
!pip install hands-on-ai



In [3]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: 3e747f47ffc4411699263cb4597f782a


## 📦 Setup and Configuration
Import required packages and setup environment.

In [11]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# The following set up code has been added
import datetime  # For handling and formatting dates
import time      # For delays, time calculations
import statistics  # For calculating averages, etc.
import pandas as pd  # If you want advanced data handling (optional)

## 🌤️ Weather Data Functions

In [9]:
# Define get_weather_data() function here
import requests
import datetime
from typing import Dict, Any

def geocode_location(location: str) -> tuple[float, float]:
    """
    Get latitude and longitude from a location name using OpenStreetMap Nominatim API.

    Args:
        location (str): Name of the city or location.

    Returns:
        tuple: (latitude, longitude) as floats.
    """
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': location,    # Query string for the location name
        'format': 'json', # Request JSON response
        'limit': 1        # Only return the top result
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an error if the request failed

    data = response.json()
    if not data:
        # If no data is returned, the location was not found
        raise ValueError(f"Location '{location}' not found.")

    # Return latitude and longitude as floats
    return float(data[0]['lat']), float(data[0]['lon'])

def fetch_weather(lat: float, lon: float, start_date: str, end_date: str) -> Dict[str, Any]:
    """
    Fetch weather data from the Open-Meteo API for the given coordinates and date range.

    Args:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        dict: JSON response from the weather API.
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': lat,
        'longitude': lon,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',  # Request these daily fields
        'current_weather': True,  # Include current weather data
        'timezone': 'auto',       # Automatically select timezone
        'start_date': start_date,
        'end_date': end_date
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an error if the request failed
    return response.json()

def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    """
    Retrieve weather data for a specified location using Open-Meteo API.

    Args:
        location (str): City or location name.
        forecast_days (int): Number of forecast days (1 to 5).

    Returns:
        dict: {
            'location': str,
            'latitude': float,
            'longitude': float,
            'current': dict,
            'forecast': list of dicts with 'date', 'temp_max', 'temp_min', 'precipitation'
        }
    """
    # Ensure forecast_days is between 1 and 5
    forecast_days = min(max(forecast_days, 1), 5)

    # Step 1: Get coordinates for the location
    lat, lon = geocode_location(location)

    # Step 2: Define the date range for the forecast
    today = datetime.date.today()
    end_date = today + datetime.timedelta(days=forecast_days - 1)

    # Step 3: Fetch weather data from the API
    weather_data = fetch_weather(lat, lon, today.isoformat(), end_date.isoformat())

    # Step 4: Organize the returned data into a structured dictionary
    result = {
        'location': location,
        'latitude': lat,
        'longitude': lon,
        'current': weather_data.get('current_weather', {}),
        'forecast': []
    }

    # Step 5: Extract daily forecast data and add to the result
    daily = weather_data.get('daily', {})
    for date, t_max, t_min, precip in zip(
        daily.get('time', []),
        daily.get('temperature_2m_max', []),
        daily.get('temperature_2m_min', []),
        daily.get('precipitation_sum', [])
    ):
        result['forecast'].append({
            'date': date,
            'temp_max': t_max,
            'temp_min': t_min,
            'precipitation': precip
        })

    # Step 6: Return the structured weather data
    return result


## 📊 Visualisation Functions

In [7]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
import matplotlib.pyplot as plt

def _extract_forecast_data(weather_data, keys):
    """
    Helper function to extract lists of values for specified keys from the forecast data.

    Args:
        weather_data (dict): The weather data dictionary, expected to have a 'forecast' key.
        keys (list of str): The keys to extract from each day's forecast.

    Returns:
        list of lists: Each sublist contains the values for one key across all forecast days.
    """
    forecast = weather_data.get('forecast', [])  # Get the list of daily forecasts
    # For each key, create a list containing that key's value from each day's forecast.
    return [[day.get(k, None) for day in forecast] for k in keys]


def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create a line plot showing max and min temperatures for each day in the forecast.

    Args:
        weather_data (dict): Must contain 'forecast' with 'date', 'temp_max', 'temp_min' for each day.
        output_type (str): 'display' to show plot, 'figure' to return the matplotlib figure object.

    Returns:
        matplotlib.figure.Figure or None: Returns the figure if requested, otherwise displays the plot.
    """
    # Extract lists of dates, max temps, and min temps from the forecast data
    dates, temp_max, temp_min = _extract_forecast_data(weather_data, ['date', 'temp_max', 'temp_min'])

    # Create a new figure and axis for the plot
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot max temperatures as a red line with circle markers
    ax.plot(dates, temp_max, label='Max Temp (°C)', marker='o', color='crimson')
    # Plot min temperatures as a blue line with circle markers
    ax.plot(dates, temp_min, label='Min Temp (°C)', marker='o', color='royalblue')
    # Fill the area between min and max temperatures for visual emphasis
    ax.fill_between(dates, temp_min, temp_max, color='lightgray', alpha=0.3)

    # Set the plot title, using the location from weather_data if available
    ax.set_title(f"Temperature Forecast for {weather_data.get('location', 'Unknown')}")
    # Label the x-axis and y-axis
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    # Add a legend to distinguish the lines
    ax.legend()
    # Add a grid for easier reading, with dashed lines and some transparency
    ax.grid(True, linestyle='--', alpha=0.5)
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    # Adjust layout to prevent label overlap
    plt.tight_layout()

    # Return the figure if requested, otherwise display the plot
    return fig if output_type == 'figure' else plt.show()


def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create a bar chart showing precipitation for each day in the forecast.

    Args:
        weather_data (dict): Must contain 'forecast' with 'date' and 'precipitation' for each day.
        output_type (str): 'display' to show plot, 'figure' to return the matplotlib figure object.

    Returns:
        matplotlib.figure.Figure or None: Returns the figure if requested, otherwise displays the plot.
    """
    # Extract lists of dates and precipitation values from the forecast data
    dates, precipitation = _extract_forecast_data(weather_data, ['date', 'precipitation'])

    # Create a new figure and axis for the plot
    fig, ax = plt.subplots(figsize=(10, 5))

    # Create a bar chart for precipitation values
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)

    # Set the plot title, using the location from weather_data if available
    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    # Label the x-axis and y-axis
    ax.set_xlabel("Date")
    ax.set_ylabel("Precipitation (mm)")
    # Add a horizontal grid for easier reading
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    # Adjust layout to prevent label overlap
    plt.tight_layout()

    # Return the figure if requested, otherwise display the plot
    return fig if output_type == 'figure' else plt.show()



In [8]:

import matplotlib.pyplot as plt

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Visualize precipitation forecast, handling missing values by imputing zeros.

    Args:
        weather_data (dict): Expected to contain a 'forecast' list with dictionaries
                             having 'date' and 'precipitation' keys.
        output_type (str):
            - 'display': Show the plot immediately.
            - 'figure': Return the matplotlib figure object for further use.

    Returns:
        matplotlib.figure.Figure or None: Returns the figure if output_type is 'figure',
                                         otherwise displays the plot and returns None.
    """
    # Retrieve the forecast list from weather_data; default to empty list if missing
    forecast = weather_data.get('forecast', [])

    # Extract dates and precipitation values from each forecast day
    # - Use a list comprehension to iterate over forecast entries
    # - For each day, get the 'date' (default to 'Unknown' if missing)
    # - For 'precipitation', if the value is None or missing, substitute 0
    # - Only include days that have a 'date' key to avoid errors
    if forecast:
        dates, precipitation = zip(*[
            (day.get('date', 'Unknown'),
             day.get('precipitation') if day.get('precipitation') is not None else 0)
            for day in forecast if 'date' in day
        ])
    else:
        # If forecast is empty, set dates and precipitation to empty tuples
        dates, precipitation = ([], [])

    # Create a new figure and axis for the bar chart with a wide aspect ratio
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot precipitation as a bar chart
    # - dates are on the x-axis
    # - precipitation amounts on the y-axis
    # - bars colored 'deepskyblue' for a visually appealing look
    # - bar width set to 0.6 for balanced spacing
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)

    # Set the title of the plot, including the location name if available
    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    # Label the x-axis as 'Date'
    ax.set_xlabel("Date")
    # Label the y-axis as 'Precipitation (mm)'
    ax.set_ylabel("Precipitation (mm)")
    # Add a horizontal grid only along the y-axis to help read precipitation values
    # - Grid lines are dashed and semi-transparent for subtlety
    ax.grid(axis='y', linestyle='--', alpha=0.5)

    # Rotate x-axis labels by 45 degrees to prevent overlap and improve readability
    plt.xticks(rotation=45)
    # Adjust layout to ensure labels and titles fit nicely within the figure
    plt.tight_layout()

    # Return the figure object if requested; otherwise, display the plot
    if output_type == 'figure':
        return fig
    else:
        plt.show()


## 🤖 Natural Language Processing

In [10]:
# Define parse_weather_question() and generate_weather_response() here
import re

def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: {
            'location': str or None,  # Extracted location or None if not found
            'time_period': str,       # Time period requested (e.g., 'today', 'tomorrow', 'next_3_days')
            'weather_attribute': str, # Weather attribute requested (e.g., 'temperature', 'precipitation')
            'is_vague': bool          # True if question is vague or missing key info
        }
    """
    original_question = question # Keep original for case-sensitive location extraction
    question = question.lower()  # Convert to lowercase for case-insensitive matching


    # Set default values for extracted information
    location = None
    time_period = 'today'  # Default time period if none specified
    weather_attribute = 'general'  # Default attribute if none specified
    is_vague = False       # Flag to indicate if the question lacks clarity

    # === TIME PERIOD ===
    if 'tomorrow' in question:
        time_period = 'tomorrow'  # User asked about tomorrow specifically
    elif 'soon' in question or 'later' in question:
        time_period = 'next_1_days'  # Interpret vague terms as next 1 day
    else:

      # Look for pattern like "next 3 days" to extract number of days
        match = re.search(r'next (\d+) days', question)
        if match:
            time_period = f"next_{int(match.group(1))}_days"

    # === WEATHER ATTRIBUTE ===
     # Check if question mentions temperature-related terms
    if any(term in question for term in ['temperature', 'temp', 'hot', 'cold']):
        weather_attribute = 'temperature'
    elif any(term in question for term in ['rain', 'precipitation', 'snow', 'wet']):
        weather_attribute = 'precipitation'
    elif 'forecast' in question:
        weather_attribute = 'forecast'
    elif 'humidity' in question:
        weather_attribute = 'humidity'
    elif 'wind' in question:
        weather_attribute = 'wind'

    # === LOCATION EXTRACTION ===
    # Look for pattern "in <location>" (e.g., "in London")
    location_match = re.search(r'\bin ([a-z\s]+)', question)
    if location_match:
        location = location_match.group(1).strip()
    else:
        # Naive fallback for capitalized location names (used in voice-to-text or GUI)
        capitalized = re.findall(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)?', original_question)
        if capitalized:
            location = ' '.join(capitalized)

    # === VAGUENESS DETECTION ===
    # Mark question as vague if location is missing or attribute is generic
    if not location or weather_attribute == 'general':
        is_vague = True

  # Return all extracted info as a dictionary

    return {
        'location': location,
        'time_period': time_period,
        'weather_attribute': weather_attribute,
        'is_vague': is_vague
    }

## 🧭 User Interface

In [11]:
# Define menu functions using pyinputplus or ipywidgets here
import re
import pyinputplus as pyip

def get_location_input():
    """
    Prompt the user to enter a valid location name.
    The location must contain only letters, spaces, or hyphens and be at least 2 characters long.

    Returns:
        str: The validated location string entered by the user.
    """

    # Define a custom validator function to check the user's input
    def location_validator(input_str):
        # Remove leading/trailing spaces for validation
        stripped_input = input_str.strip()

        # Check that input contains only letters (a-z, A-Z), spaces, or hyphens
        if not re.match(r'^[a-zA-Z\s\-]+$', stripped_input):
            # Raise an error if invalid characters are found
            raise ValueError("Location must contain only letters, spaces or hyphens.")

        # Check that the input length is at least 2 characters after stripping
        if len(stripped_input) < 2:
            # Raise an error if the location name is too short
            raise ValueError("Location name is too short.")

        # If all checks pass, return the cleaned input
        return stripped_input

    # Use pyinputplus to prompt the user for input, applying the custom validator
    # The prompt message guides the user on acceptable input format
    return pyip.inputStr(
        prompt="Enter the city or location name (e.g., Perth, New York): ",
        applyFunc=location_validator
    )



## 🧩 Main Application Logic

In [12]:
# Tie everything together here

# Function 1- Weather Data Functions

# Define get_weather_data() function here
import requests
import datetime
from typing import Dict, Any

def geocode_location(location: str) -> tuple[float, float]:
    """
    Get latitude and longitude from a location name using OpenStreetMap Nominatim API.

    Args:
        location (str): Name of the city or location.

    Returns:
        tuple: (latitude, longitude) as floats.
    """
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': location,    # Query string for the location name
        'format': 'json', # Request JSON response
        'limit': 1        # Only return the top result
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an error if the request failed

    data = response.json()
    if not data:
        # If no data is returned, the location was not found
        raise ValueError(f"Location '{location}' not found.")

    # Return latitude and longitude as floats
    return float(data[0]['lat']), float(data[0]['lon'])

def fetch_weather(lat: float, lon: float, start_date: str, end_date: str) -> Dict[str, Any]:
    """
    Fetch weather data from the Open-Meteo API for the given coordinates and date range.

    Args:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        dict: JSON response from the weather API.
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': lat,
        'longitude': lon,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',  # Request these daily fields
        'current_weather': True,  # Include current weather data
        'timezone': 'auto',       # Automatically select timezone
        'start_date': start_date,
        'end_date': end_date
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an error if the request failed
    return response.json()

def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    """
    Retrieve weather data for a specified location using Open-Meteo API.

    Args:
        location (str): City or location name.
        forecast_days (int): Number of forecast days (1 to 5).

    Returns:
        dict: {
            'location': str,
            'latitude': float,
            'longitude': float,
            'current': dict,
            'forecast': list of dicts with 'date', 'temp_max', 'temp_min', 'precipitation'
        }
    """
    # Ensure forecast_days is between 1 and 5
    forecast_days = min(max(forecast_days, 1), 5)

    # Step 1: Get coordinates for the location
    lat, lon = geocode_location(location)

    # Step 2: Define the date range for the forecast
    today = datetime.date.today()
    end_date = today + datetime.timedelta(days=forecast_days - 1)

    # Step 3: Fetch weather data from the API
    weather_data = fetch_weather(lat, lon, today.isoformat(), end_date.isoformat())

    # Step 4: Organize the returned data into a structured dictionary
    result = {
        'location': location,
        'latitude': lat,
        'longitude': lon,
        'current': weather_data.get('current_weather', {}),
        'forecast': []
    }

    # Step 5: Extract daily forecast data and add to the result
    daily = weather_data.get('daily', {})
    for date, t_max, t_min, precip in zip(
        daily.get('time', []),
        daily.get('temperature_2m_max', []),
        daily.get('temperature_2m_min', []),
        daily.get('precipitation_sum', [])
    ):
        result['forecast'].append({
            'date': date,
            'temp_max': t_max,
            'temp_min': t_min,
            'precipitation': precip
        })

    # Step 6: Return the structured weather data
    return result

 #-----------------------------------------

 # Function 2- Vusualisation function

 # Define create_temperature_visualisation() and create_precipitation_visualisation() here
import matplotlib.pyplot as plt

def _extract_forecast_data(weather_data, keys):
    """
    Helper function to extract lists of values for specified keys from the forecast data.

    Args:
        weather_data (dict): The weather data dictionary, expected to have a 'forecast' key.
        keys (list of str): The keys to extract from each day's forecast.

    Returns:
        list of lists: Each sublist contains the values for one key across all forecast days.
    """
    forecast = weather_data.get('forecast', [])  # Get the list of daily forecasts
    # For each key, create a list containing that key's value from each day's forecast.
    return [[day.get(k, None) for day in forecast] for k in keys]


def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create a line plot showing max and min temperatures for each day in the forecast.

    Args:
        weather_data (dict): Must contain 'forecast' with 'date', 'temp_max', 'temp_min' for each day.
        output_type (str): 'display' to show plot, 'figure' to return the matplotlib figure object.

    Returns:
        matplotlib.figure.Figure or None: Returns the figure if requested, otherwise displays the plot.
    """
    # Extract lists of dates, max temps, and min temps from the forecast data
    dates, temp_max, temp_min = _extract_forecast_data(weather_data, ['date', 'temp_max', 'temp_min'])

    # Create a new figure and axis for the plot
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot max temperatures as a red line with circle markers
    ax.plot(dates, temp_max, label='Max Temp (°C)', marker='o', color='crimson')
    # Plot min temperatures as a blue line with circle markers
    ax.plot(dates, temp_min, label='Min Temp (°C)', marker='o', color='royalblue')
    # Fill the area between min and max temperatures for visual emphasis
    ax.fill_between(dates, temp_min, temp_max, color='lightgray', alpha=0.3)

    # Set the plot title, using the location from weather_data if available
    ax.set_title(f"Temperature Forecast for {weather_data.get('location', 'Unknown')}")
    # Label the x-axis and y-axis
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    # Add a legend to distinguish the lines
    ax.legend()
    # Add a grid for easier reading, with dashed lines and some transparency
    ax.grid(True, linestyle='--', alpha=0.5)
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    # Adjust layout to prevent label overlap
    plt.tight_layout()

    # Return the figure if requested, otherwise display the plot
    return fig if output_type == 'figure' else plt.show()


def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create a bar chart showing precipitation for each day in the forecast.

    Args:
        weather_data (dict): Must contain 'forecast' with 'date' and 'precipitation' for each day.
        output_type (str): 'display' to show plot, 'figure' to return the matplotlib figure object.

    Returns:
        matplotlib.figure.Figure or None: Returns the figure if requested, otherwise displays the plot.
    """
    # Extract lists of dates and precipitation values from the forecast data
    dates, precipitation = _extract_forecast_data(weather_data, ['date', 'precipitation'])

    # Create a new figure and axis for the plot
    fig, ax = plt.subplots(figsize=(10, 5))

    # Create a bar chart for precipitation values
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)

    # Set the plot title, using the location from weather_data if available
    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    # Label the x-axis and y-axis
    ax.set_xlabel("Date")
    ax.set_ylabel("Precipitation (mm)")
    # Add a horizontal grid for easier reading
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    # Adjust layout to prevent label overlap
    plt.tight_layout()

    # Return the figure if requested, otherwise display the plot
    return fig if output_type == 'figure' else plt.show()
    import matplotlib.pyplot as plt

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Visualize precipitation forecast, handling missing values by imputing zeros.

    Args:
        weather_data (dict): Expected to contain a 'forecast' list with dictionaries
                             having 'date' and 'precipitation' keys.
        output_type (str):
            - 'display': Show the plot immediately.
            - 'figure': Return the matplotlib figure object for further use.

    Returns:
        matplotlib.figure.Figure or None: Returns the figure if output_type is 'figure',
                                         otherwise displays the plot and returns None.
    """
    # Retrieve the forecast list from weather_data; default to empty list if missing
    forecast = weather_data.get('forecast', [])

    # Extract dates and precipitation values from each forecast day
    # - Use a list comprehension to iterate over forecast entries
    # - For each day, get the 'date' (default to 'Unknown' if missing)
    # - For 'precipitation', if the value is None or missing, substitute 0
    # - Only include days that have a 'date' key to avoid errors
    if forecast:
        dates, precipitation = zip(*[
            (day.get('date', 'Unknown'),
             day.get('precipitation') if day.get('precipitation') is not None else 0)
            for day in forecast if 'date' in day
        ])
    else:
        # If forecast is empty, set dates and precipitation to empty tuples
        dates, precipitation = ([], [])

    # Create a new figure and axis for the bar chart with a wide aspect ratio
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot precipitation as a bar chart
    # - dates are on the x-axis
    # - precipitation amounts on the y-axis
    # - bars colored 'deepskyblue' for a visually appealing look
    # - bar width set to 0.6 for balanced spacing
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)

    # Set the title of the plot, including the location name if available
    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    # Label the x-axis as 'Date'
    ax.set_xlabel("Date")
    # Label the y-axis as 'Precipitation (mm)'
    ax.set_ylabel("Precipitation (mm)")
    # Add a horizontal grid only along the y-axis to help read precipitation values
    # - Grid lines are dashed and semi-transparent for subtlety
    ax.grid(axis='y', linestyle='--', alpha=0.5)

    # Rotate x-axis labels by 45 degrees to prevent overlap and improve readability
    plt.xticks(rotation=45)
    # Adjust layout to ensure labels and titles fit nicely within the figure
    plt.tight_layout()

    # Return the figure object if requested; otherwise, display the plot
    if output_type == 'figure':
        return fig
    else:
        plt.show()

 #-----------------------------------------

 # Function 3- Natural language processing

 # Define parse_weather_question() and generate_weather_response() here
import re

def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: {
            'location': str or None,  # Extracted location or None if not found
            'time_period': str,       # Time period requested (e.g., 'today', 'tomorrow', 'next_3_days')
            'weather_attribute': str, # Weather attribute requested (e.g., 'temperature', 'precipitation')
            'is_vague': bool          # True if question is vague or missing key info
        }
    """
    original_question = question # Keep original for case-sensitive location extraction
    question = question.lower()  # Convert to lowercase for case-insensitive matching


    # Set default values for extracted information
    location = None
    time_period = 'today'  # Default time period if none specified
    weather_attribute = 'general'  # Default attribute if none specified
    is_vague = False       # Flag to indicate if the question lacks clarity

    # === TIME PERIOD ===
    if 'tomorrow' in question:
        time_period = 'tomorrow'  # User asked about tomorrow specifically
    elif 'soon' in question or 'later' in question:
        time_period = 'next_1_days'  # Interpret vague terms as next 1 day
    else:

      # Look for pattern like "next 3 days" to extract number of days
        match = re.search(r'next (\d+) days', question)
        if match:
            time_period = f"next_{int(match.group(1))}_days"

    # === WEATHER ATTRIBUTE ===
     # Check if question mentions temperature-related terms
    if any(term in question for term in ['temperature', 'temp', 'hot', 'cold']):
        weather_attribute = 'temperature'
    elif any(term in question for term in ['rain', 'precipitation', 'snow', 'wet']):
        weather_attribute = 'precipitation'
    elif 'forecast' in question:
        weather_attribute = 'forecast'
    elif 'humidity' in question:
        weather_attribute = 'humidity'
    elif 'wind' in question:
        weather_attribute = 'wind'

    # === LOCATION EXTRACTION ===
    # Look for pattern "in <location>" (e.g., "in London")
    location_match = re.search(r'\bin ([a-z\s]+)', question)
    if location_match:
        location = location_match.group(1).strip()
    else:
        # Naive fallback for capitalized location names (used in voice-to-text or GUI)
        capitalized = re.findall(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)?', original_question)
        if capitalized:
            location = ' '.join(capitalized)

    # === VAGUENESS DETECTION ===
    # Mark question as vague if location is missing or attribute is generic
    if not location or weather_attribute == 'general':
        is_vague = True

  # Return all extracted info as a dictionary

    return {
        'location': location,
        'time_period': time_period,
        'weather_attribute': weather_attribute,
        'is_vague': is_vague
    }

#-----------------------------------------

# Function 4- User Interface

# Define menu functions using pyinputplus or ipywidgets here
import re


def get_location_input():
    """
    Prompt the user to enter a valid location name.
    The location must contain only letters, spaces, or hyphens and be at least 2 characters long.

    Returns:
        str: The validated location string entered by the user.
    """

    # Define a custom validator function to check the user's input
    def location_validator(input_str):
        # Remove leading/trailing spaces for validation
        stripped_input = input_str.strip()

        # Check that input contains only letters (a-z, A-Z), spaces, or hyphens
        if not re.match(r'^[a-zA-Z\s\-]+$', stripped_input):
            # Raise an error if invalid characters are found
            raise ValueError("Location must contain only letters, spaces or hyphens.")

        # Check that the input length is at least 2 characters after stripping
        if len(stripped_input) < 2:
            # Raise an error if the location name is too short
            raise ValueError("Location name is too short.")

        # If all checks pass, return the cleaned input
        return stripped_input

    # Use pyinputplus to prompt the user for input, applying the custom validator
    # The prompt message guides the user on acceptable input format
    return pyip.inputStr(
        prompt="Enter the city or location name (e.g., Perth, New York): ",
        applyFunc=location_validator
    )

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

#-----------------------------------------
# Function 1- Weather Data Functions
#-----------------------------------------

# 1-Define get Weather data

def get_weather_data(location: str, forecast_days: int = 5) -> dict:
    """
    >>> data = get_weather_data("Perth", 3)
    >>> 'current' in data and 'forecast' in data
    True
    >>> len(data['forecast']) == 3
    True
    """
    # full function as I've written

# 2- Run doctest
import doctest
doctest.testmod()

# 3- Run unittest
import unittest

class TestWeatherData(unittest.TestCase):
    def test_valid_location(self):
        data = get_weather_data("Perth", 3)
        self.assertIn('current', data)
        self.assertEqual(len(data['forecast']), 3)

    def test_forecast_day_limit(self):
        data = get_weather_data("Perth", 10)
        self.assertLessEqual(len(data['forecast']), 5)

    def test_invalid_location(self):
        with self.assertRaises(ValueError):
            get_weather_data("NonsenseCityNameThatDoesNotExist")

unittest.main(argv=[''], verbosity=2, exit=False)

# 4- Simulate pytest-style tests
def test_get_weather_data():
    data = get_weather_data("Perth", 2)
    assert 'forecast' in data, "Missing forecast key"
    assert len(data['forecast']) == 2, "Forecast should be 2 days"

    try:
        get_weather_data("FakePlaceShouldFail123")
    except ValueError:
        pass
    else:
        raise AssertionError("Expected ValueError for invalid location")

test_get_weather_data()
print("All pytest-style tests passed.")

# 5- Debugging a List of Cities
def debug_weather_cities(city_list):
    for city in city_list:
        print(f"Checking weather for: {city}")
        try:
            data = get_weather_data(city, 2)
            print(f"{city} OK — Current Temp: {data['current'].get('temperature', 'N/A')}°C")
        except Exception as e:
            print(f"{city} FAILED — {e}")

debug_weather_cities(["Perth", "Sydney", "Atlantis", "1234"])

#-----------------------------------------
# Function 2- Visualisation Functions
#-----------------------------------------

# 1-Define & Test create_temperature_visualisation() and create_precipitation_visualisation()
import matplotlib.pyplot as plt

def create_temperature_visualisation(weather_data, output_type='display'):
    """
    >>> sample_data = {
    ...     "location": "Testville",
    ...     "forecast": [
    ...         {"date": "2024-01-01", "temp_max": 30, "temp_min": 20},
    ...         {"date": "2024-01-02", "temp_max": 32, "temp_min": 21},
    ...     ]
    ... }
    >>> fig = create_temperature_visualisation(sample_data, output_type='figure')
    >>> isinstance(fig, plt.Figure)
    True
    """
    # Function body as written above
def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    >>> sample_data = {
    ...     "location": "RainCity",
    ...     "forecast": [
    ...         {"date": "2024-01-01", "precipitation": 4.5},
    ...         {"date": "2024-01-02", "precipitation": None},
    ...     ]
    ... }
    >>> fig = create_precipitation_visualisation(sample_data, output_type='figure')
    >>> isinstance(fig, plt.Figure)
    True
    """
    # Function body as written above

# 2- Run doctest
import doctest
doctest.testmod()

# 3- Run unittest
import unittest

class TestWeatherVisualisation(unittest.TestCase):

    def setUp(self):
        self.temp_data = {
            "location": "TestTown",
            "forecast": [
                {"date": "2024-01-01", "temp_max": 28, "temp_min": 19},
                {"date": "2024-01-02", "temp_max": 29, "temp_min": 18}
            ]
        }
        self.precip_data = {
            "location": "DrizzleVille",
            "forecast": [
                {"date": "2024-01-01", "precipitation": 2.0},
                {"date": "2024-01-02", "precipitation": None}
            ]
        }

    def test_temp_returns_figure(self):
        fig = create_temperature_visualisation(self.temp_data, output_type='figure')
        self.assertIsInstance(fig, plt.Figure)

    def test_precip_returns_figure(self):
        fig = create_precipitation_visualisation(self.precip_data, output_type='figure')
        self.assertIsInstance(fig, plt.Figure)

unittest.main(argv=[''], verbosity=2, exit=False)

# 4- Simulate pytest-style Tests
def test_visualisation_outputs():
    test_temp = {
        "location": "SimTown",
        "forecast": [
            {"date": "2024-01-01", "temp_max": 27, "temp_min": 17},
            {"date": "2024-01-02", "temp_max": 30, "temp_min": 18}
        ]
    }

    test_precip = {
        "location": "WetVille",
        "forecast": [
            {"date": "2024-01-01", "precipitation": 3.5},
            {"date": "2024-01-02", "precipitation": None}
        ]
    }

    assert isinstance(create_temperature_visualisation(test_temp, output_type='figure'), plt.Figure), "Temp figure failed"
    assert isinstance(create_precipitation_visualisation(test_precip, output_type='figure'), plt.Figure), "Precip figure failed"

test_visualisation_outputs()
print("All pytest-style visualisation tests passed.")

# 5- Debugging a Visualisation Run
def debug_weather_graphs():
    sample = {
        "location": "DebugTown",
        "forecast": [
            {"date": "2024-01-01", "temp_max": 31, "temp_min": 21, "precipitation": 0.0},
            {"date": "2024-01-02", "temp_max": 33, "temp_min": 22, "precipitation": None}
        ]
    }
    print("Generating temperature plot...")
    create_temperature_visualisation(sample)
    print("Generating precipitation plot...")
    create_precipitation_visualisation(sample)

debug_weather_graphs()

#-----------------------------------------
# Function 3- Natural Language Processing
#-----------------------------------------

# 1- Define parse_weather_question() with doctests
def parse_weather_question(question):
    """
    >>> parse_weather_question("Will it rain in Sydney tomorrow?")
    {'location': 'sydney', 'time_period': 'tomorrow', 'weather_attribute': 'precipitation', 'is_vague': False}

    >>> parse_weather_question("Hot in Tokyo?")
    {'location': 'Tokyo', 'time_period': 'today', 'weather_attribute': 'temperature', 'is_vague': False}

    >>> parse_weather_question("Will it rain soon?")
    {'location': None, 'time_period': 'next_1_days', 'weather_attribute': 'precipitation', 'is_vague': True}
    """
    # Function body as I've written

# 2-  Run doctest
import doctest
doctest.testmod()

# 3- Run unittest
import unittest

class TestParseWeatherQuestion(unittest.TestCase):

    def test_full_question(self):
        result = parse_weather_question("Will it rain in Sydney tomorrow?")
        self.assertEqual(result['location'], 'sydney')
        self.assertEqual(result['time_period'], 'tomorrow')
        self.assertEqual(result['weather_attribute'], 'precipitation')
        self.assertFalse(result['is_vague'])

    def test_vague_question(self):
        result = parse_weather_question("Will it rain soon?")
        self.assertEqual(result['time_period'], 'next_1_days')
        self.assertEqual(result['weather_attribute'], 'precipitation')
        self.assertTrue(result['is_vague'])

    def test_capitalized_location(self):
        result = parse_weather_question("Is it hot in Tokyo?")
        self.assertEqual(result['location'], 'tokyo')
        self.assertEqual(result['weather_attribute'], 'temperature')

unittest.main(argv=[''], verbosity=2, exit=False)

# 4- Simulate pytest-style Tests
def test_parse_weather_question():
    assert parse_weather_question("Forecast in Berlin")['weather_attribute'] == 'forecast'
    assert parse_weather_question("Will it snow in Toronto in 3 days?")['weather_attribute'] == 'precipitation'
    assert parse_weather_question("Weather soon?")['is_vague'] == True
    assert parse_weather_question("Will it rain in Paris for the next 3 days?")['time_period'] == 'next_3_days'

test_parse_weather_question()
print("All pytest-style NLP tests passed.")

# 5- Debugging a List of User Questions
def debug_question_list(questions):
    for q in questions:
        print(f"\nQuestion: {q}")
        result = parse_weather_question(q)
        print(f"→ Parsed: {result}")

debug_question_list([
    "Will it rain in Melbourne tomorrow?",
    "Hot in Brisbane?",
    "Weather in Osaka next 3 days?",
    "Humidity?",
    "Is it windy soon?"
])

#-----------------------------------------
# Function 4- User Interface
#-----------------------------------------

# 1- Define get_location_input() and its validator (with doctests)
import re
import pyinputplus as pyip

def get_location_input():
    """
    Prompt for a valid city name.

    Returns:
        str: Cleaned city name

    >>> location_validator("Perth")
    'Perth'
    >>> location_validator("New York")
    'New York'
    >>> location_validator("R")
    Traceback (most recent call last):
        ...
    ValueError: Location name is too short.
    >>> location_validator("!@#")
    Traceback (most recent call last):
        ...
    ValueError: Location must contain only letters, spaces or hyphens.
    """
    return pyip.inputStr(
        prompt="Enter the city or location name (e.g., Perth, New York): ",
        applyFunc=location_validator
    )

def location_validator(input_str):
    stripped_input = input_str.strip()

    if not re.match(r'^[a-zA-Z\s\-]+$', stripped_input):
        raise ValueError("Location must contain only letters, spaces or hyphens.")

    if len(stripped_input) < 2:
        raise ValueError("Location name is too short.")

    return stripped_input

# 2- Run doctest
import doctest
doctest.testmod()

# 3- Run unittest
import unittest

class TestLocationValidator(unittest.TestCase):

    def test_valid_location(self):
        self.assertEqual(location_validator("New York"), "New York")
        self.assertEqual(location_validator("Rio-de-Janeiro"), "Rio-de-Janeiro")

    def test_invalid_symbols(self):
        with self.assertRaises(ValueError):
            location_validator("Paris123")

    def test_too_short(self):
        with self.assertRaises(ValueError):
            location_validator("A")

    def test_whitespace_only(self):
        with self.assertRaises(ValueError):
            location_validator("   ")

unittest.main(argv=[''], verbosity=2, exit=False)

# 4- Simulate pytest-style Tests
def test_location_validator():
    assert location_validator("Tokyo") == "Tokyo"
    assert location_validator("Los Angeles") == "Los Angeles"
    try:
        location_validator("X")
    except ValueError as e:
        assert str(e) == "Location name is too short."
    try:
        location_validator("Berlin123")
    except ValueError as e:
        assert str(e) == "Location must contain only letters, spaces or hyphens."

test_location_validator()
print("All pytest-style location input tests passed.")

# 5-  Debugging a List of User Inputs
def debug_location_inputs(test_inputs):
    for input_val in test_inputs:
        print(f"Input: {input_val}")
        try:
            result = location_validator(input_val)
            print(f"✅ Validated: {result}")
        except ValueError as e:
            print(f"❌ Error: {e}")

debug_location_inputs(["Perth", "NY", "123", "X", "Paris!", "Rio de Janeiro", " "])

#-----------------------------------------------------------------------------------------------------------------







## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.